In [111]:
import numpy as np;
from math import sin, cos, sqrt, atan2, radians,inf;

class Autocar:   
    def __init__(self,AC_location = 0,state_of_charge = -1,min_lat = 52,
                 min_long = 12.9,capacity = 75, powerpdist = 14):
        #map constraint
        self.AC_location = np.random.random(2) + (min_lat,min_long)
        self.min_lat = min_lat;
        self.min_long = min_long;
        # car representet as random distribution funktion --> random langitude and latidude
        #AC_long = np.random.random(1);
        #AC_lat = np.random.random(1);
        # alternative: random function generates random vector that represent a car with only 20% of charge
        # random number amount between 0 and 1
        # car needs parameters
        self.capacity  = capacity; # example for Tessla S modal, BMWi3 ; smart60kW fortwo 17,6kWh
        #possible_distance = 400 # Tessla s Modal; 260km BMWi3 (daily distance)
        #Stromverbrauch in kWh/100 km: 14,6 - 14,0
        self.powerpdist = powerpdist;#power per distance --> kWh per 100km
        #powtodist = powerpdist/100; #power [kWh] need for 1km; theoretically something that might be measured in the car andoptimized by the car itself
        self.powerkm = powerpdist/100
        if state_of_charge < 0:    
            self.state_of_charge = np.random.random(1); # state of battery between 0 and 1 --> if it is smaller 0.2 it will check where it can charge its battery
        else:
            self.state_of_charge = state_of_charge;
            
    def show_attributes(self):
        print('location:',self.AC_location,' (min_lat:', self.min_lat,'min_long:', self.min_long,')\ncapacity [kWh]:',
              self.capacity,'\npower consumption per km:', self.powerkm,'\nstate of charge [% of capacity]:',self.state_of_charge*100 )

    def find_distance_km(self, lat1,lng1,lat2,lng2):
        EARTH_RADIUS = 6373.0
        lat1 = radians(lat1)
        lng1 = radians(lng1)
        lat2 = radians(lat2)
        lng2 = radians(lng2)
        
        dlon = lng2 - lng1
        dlat = lat2 - lat1
        
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        
        return EARTH_RADIUS * c 


In [121]:
import random
import pandas as pd

min_lat = 52
min_long = 12.9
grids_price  = []
grids_loc = []
ac = []
acs = []
c_to_grid = []
energy_deficit = []

### create 10 grids, 10 cars

for x in range(6):
    grids_loc.append(np.random.random(2) + (min_lat,min_long))
    acs.append(Autocar(capacity = random.uniform(50,120), powerpdist = random.uniform(5, 20)))
    ac.append([random.uniform(50,120),  random.uniform(5, 20), np.random.random(2) + (min_lat,min_long)])
    
    ### calculate total energy to grid considering distance
    dist = acs[x].find_distance_km(grids_loc[0][0], grids_loc[0][1], acs[x].AC_location[0], acs[x].AC_location[1])
    c_to_grid = acs[x].powerkm*dist
    ### consider only reachable grids
    if (c_to_grid <= (acs[x].state_of_charge*acs[x].capacity)):
        ### add the energy needed to reach grid to the total deficit
        total_charge = c_to_grid+(acs[x].capacity-(acs[x].state_of_charge*acs[x].capacity))
        energy_deficit.append(total_charge[0])
        grids_price.append(random.uniform(0,1))


#############################
### linear Programming#######
#############################

from pulp import * 

prob = LpProblem("AC", LpMinimize)

### every x represents a boolean variable wether to pick a grid

grid_1 = LpVariable("grid 1",0, 1, LpInteger)
grid_2 = LpVariable("grid 2",0, 1, LpInteger)
grid_3 = LpVariable("grid 3",0, 1, LpInteger)


### OBJECTIVE FUNCTION
prob += energy_deficit[0]*(grids_price[0]*grid_1+grids_price[1]*grid_2+grids_price[2]*grid_3)

### CONSTRAINTS
# all binary decision variables should sum up to 1
prob += grid_1+grid_2+grid_3 == 1

prob.writeLP("AC.lp")
prob.solve()

#print("grid 1: %d , grid 2: %d , grid 3: %d " % (value(grid_1), value(grid_2), value(grid_3)))
val = [grid_1, grid_2, grid_3]

for x in val:
    if(value(x)==1):
        print("Lowest cost at..", x)
        print("Price: ", value(prob.objective))


Lowest cost at.. grid_3
Price:  30.101031308582165
